# Barrier swaption

In this section we consider Monte Carlo evaluation of a knock-out swaption
under the LMM. We use the knock-out swaption as a guide in our exposition,
its treatment is rather general and it can be used to value diﬀerent barrier
options, where the underlying and barrier can be expressed as functionals of
some diﬀusion process.

A European payer (receiver) swaption is an option that gives its holder a
right, but not an obligation, to enter a payer (receiver) swap at a future date
at a given fixed rate K. Usually, the swaption maturity coincides with the first
reset date T0 of the underlying swap. The underlying swap length TN− T0 is
called the tenor of the swaption.

$$
V_{\text{swaption}}(0) = P(0, T_0) \, \mathbb{E}^{\mathbb{Q}^{T_0}} \left[
\delta \left( R_{\text{swap}}(T_0) - K \right)_+
\sum_{j=1}^{N} P(T_0, T_j) \, \chi(\theta > T_0)
\right]
\tag{6.1}
$$

where $\theta$ is the first exit time of the process $R_{\text{swap}}(s), s \geq 0$,
from the interval $(0, R_{\text{up}})$.
That is, $\theta$ is the earliest time at which the swap rate crosses the upper barrier $R_{\text{up}}$,
causing the knock-out feature of the swaption to be triggered.

In [207]:
import numpy as np
from scipy.linalg import cholesky
from scipy.optimize import minimize
from tqdm import tqdm
from scipy.stats import norm

## 0. Setup du modèle

- Nombre de forward rates `N`
- Maturité $(T_0)$ découpée en `M` pas de taille `h`
- Courbe initiale `L0`, volatilités `sigma`
- Corrélation exponentielle $( \rho_{ij} = e^{-\beta|T_i - T_j|} $)

In [208]:
# Paramètres
N     = 10
T0    = 10.0
delta = 1.0
M     = 100
h     = T0 / M
K     = 0.01   # Strike
Rup   = 0.075  # Barrier
L0    = 0.05 * np.ones(N)
sigma = 0.10 * np.ones(N)
beta  = 0.1
T_star= T0 + delta * N

$$
\rho_{i,j} = \exp(-\beta |T_i - T_j|).
\tag{3.6}
$$

Where $U$ is the upper triangular matrix
$$
\rho = UU^\top,
\tag{3.5}
$$

In [209]:
# Corrélation et Cholesky
times = T0 + delta * np.arange(N)       # T_i = T0, T0+δ, …, T0+(N-1)δ
rho   = np.exp(-beta * np.abs(times[:, None] - times[None, :]))
U     = cholesky(rho, lower=False)

# PRICE conventionnel
T0_index = int(T0 / delta)
P0_T0 = np.prod(1.0 / (1 + delta * L0[:T0_index]))

# Matrice précomputée pour drift (i >= j)
full_mat  = rho * np.outer(sigma, sigma)
drift_mat = np.tril(full_mat)           # (i,j) zero if j>i

### Swap Rate Function

The swap rate $R_{swap}(s)$ can be expressed in terms of the spanning LIBOR rates as:

$$
R_{\text{swap}}(s) = \frac{1 - \frac{1}{\prod\limits_{j=0}^{N-1} \left(1 + \delta L^j(s)\right)}}
{\delta \sum\limits_{i=0}^{N-1} \frac{1}{\prod\limits_{j=0}^{i} \left(1 + \delta L^j(s)\right)}}.
\tag{6.2}
$$

In [210]:
def R_swap(logL, delta):
    L = np.exp(logL)
    discount_prod = np.prod(1 + delta * L)
    numer = 1 - 1.0 / discount_prod
    denom = delta * np.sum(1.0 / np.cumprod(1 + delta * L))
    return numer / denom

## 1. Approximation analytique

For test purposes, let us introduce an analytical approximation for the barrier swaption. To this end, we note that under the Swap Market Model (SMM, see details in [4, 18, 21]) the barrier swaption pricing problem admits the closed-form solution (cf. (4.3)):


- La payer swaption KO est deep in-the-money
- La **receiver swaption KO** est deep out-of-the-money

In [211]:
# Swap rate initial R0
R0 = R_swap(np.log(L0), delta)

# --- 4) Discount factors P(0,Tj) pour j=1..N ---
P0_T = np.concatenate(([1.0], np.cumprod(1.0 / (1 + delta * L0))))
annuity = np.sum(P0_T[1:])   # somme des P(0,Tj)

# --- 5) Pondérations ω_i(0) vectorisées ---
prefix = np.ones(N)
prefix[1:] = np.cumprod(1 + delta * L0[:-1])
denom_w = delta * np.sum(1.0 / prefix)
weights = (1.0 - 1.0 / prefix) / denom_w       # shape (N,)

# --- 6) Volatilité effective v selon Rebonato (6.5) ---
# ∫₀^T0 σ_i(s)σ_j(s) ds = σ_i * σ_j * T0
integrals = np.outer(sigma, sigma) * T0         # matrice (i,j)
wL = weights * L0                              # vecteur (i)
num = np.outer(wL, wL) * rho * integrals        # numérateur v²
v2  = num.sum() / (R0**2)
v   = np.sqrt(v2)

# --- 7) Fonctions d± ---
dplus  = lambda x: (np.log(x) + 0.5 * v**2) / v
dminus = lambda x: (np.log(x) - 0.5 * v**2) / v

# --- 7. Ratios utiles ---
x1 = R0 / K
x2 = R0 / Rup
y1 = Rup**2 / (K * R0)
y2 = Rup / R0

# --- 8. Termes de la formule fermée (eq. 6.4) ---
term1 = R0 * (norm.cdf(dplus(x1)) - norm.cdf(dplus(x2)))
term3 = - K   * (norm.cdf(dminus(x1)) - norm.cdf(dminus(x2)))
term4 = - Rup * (norm.cdf(dplus(y1)) - norm.cdf(dplus(y2)))
term5 =   K * R0 / Rup * (norm.cdf(dminus(y1)) - norm.cdf(dminus(y2)))

# --- 9. Prix analytique final (payer swaption barrière) ---
brace = term1 + term3 + term4 + term5
price_payer = delta * annuity * brace

# --- 10. Receiver swaption KO (≈ moitié du payer) ---
price_receiver = 0.5 * price_payer

# --- Affichage ---
print(f"Swap rate initial R0:          {R0:.6f}")
print(f"Volatilité effective v:        {v:.6f}")
print(f"Prix analytique payer KO :     {price_payer:.6f}")
print(f"Prix analytique receiver KO ≈  {price_receiver:.6f}")

Swap rate initial R0:          0.050000
Volatilité effective v:        0.065629
Prix analytique payer KO :     0.308869
Prix analytique receiver KO ≈  0.154435


## 2. Projection sur la surface barrière

For completeness of the exposition, let us discuss how the projection $\ln L_k^{\pi}$
can be simulated before we return to the description of the algorithm. The
problem of finding point $\ln L_k^{\pi}$ is equivalent to finding the minimum value of the function

$$
\left| \ln L_k^\pi - \ln L_k \right|^2 = \left( \ln L_k^{\pi,0} - \ln L_k^0 \right)^2 + \cdots + \left( \ln L_k^{\pi,N-1} - \ln L_k^{N-1} \right)^2 \tag{6.11}
$$
subject to the constraint
$$
\ln \left(
\frac{ \displaystyle\prod_{j=0}^{N-1} \left( 1 + \delta L_k^{\pi,j} \right) - 1 }{
\delta \left( 1 + \displaystyle\sum_{i=0}^{N-2} \prod_{j=i+1}^{N-1} \left( 1 + \delta L_k^{\pi,j} \right) \right) }
\right) = \ln R_{\text{up}} \tag{6.12}
$$

In [212]:
## Full constrained optimization

def project_to_barrier(x, delta, Rup):
    cons = {
        'type': 'eq',
        'fun': lambda logL: np.log(R_swap(logL, delta)) - np.log(Rup)
    }
    res = minimize(
        fun=lambda logL: np.sum((logL - x)**2),
        x0=x.copy(),
        constraints=cons,
        method='SLSQP'
    )
    if not res.success:
        raise RuntimeError("Projection failed: " + res.message)
    return res.x

Hence the minimization problem is reduced to finding the point $\ln L_k^{\pi,1}, \ldots, \ln L_k^{\pi,N-1}$ at which the function $|\ln L_k^{\pi} - \ln L_k|^2$ from (6.11) with $\ln L_k^{\pi,0}$ from (6.13) has its minimum value. This optimization problem can be solved using standard procedures, e.g. the MATLAB function lsqnonlin().


The equation (6.13) is given by:
$$
\ln L_k^{\pi,0} = \ln \left( \frac{
R_{up} \cdot \left( 1 + \sum_{i=0}^{N-2} \prod_{j=i+1}^{N-1} \left( 1 + \delta L_k^{\pi,j} \right) \right) + 1
}{
\prod_{j=1}^{N-1} \left( 1 + \delta L_k^{\pi,j} \right)
} - \frac{1}{\delta} \right) \tag{6.13}
$$

In [213]:
def project_to_barrier2(x, delta, Rup):
    """
    Fast and robust projection of logL onto the surface R_swap = Rup.
    Uses semi-analytic formula for logL0 (eq. 6.13) to reduce to (N-1) variables.
    """
    N = x.shape[0]
    eps = 1e-6  # ← internal safeguard threshold

    def objective(y):
        try:
            L_tail = np.exp(y)
            one_plus = 1 + delta * L_tail
            revcp = np.cumprod(one_plus[::-1])[::-1]
            prods = revcp
            sum_term = 1 + np.sum(prods)

            numerator = Rup * sum_term + 1
            denominator = np.prod(one_plus)
            term = numerator / denominator

            log_arg = term - 1.0 / delta
            if log_arg <= eps:
                return 1e6 * (eps - log_arg)**2

            logL0 = np.log(log_arg)

            logLpi = np.empty(N)
            logLpi[0] = logL0
            logLpi[1:] = y
            return np.sum((logLpi - x)**2)

        except Exception:
            return 1e12

    y0 = np.clip(x[1:], -5.0, 5.0)

    res = minimize(objective, y0, method='L-BFGS-B',
                   options={'ftol': 1e-9, 'gtol': 1e-5, 'maxiter': 100})

    if not res.success:
        raise RuntimeError(f"Projection2 failed: {res.message}")

    # Reconstruct final logLpi
    y_opt = res.x
    L_tail = np.exp(y_opt)
    one_plus = 1 + delta * L_tail
    revcp = np.cumprod(one_plus[::-1])[::-1]
    prods = revcp
    sum_term = 1 + np.sum(prods)
    numerator = Rup * sum_term + 1
    denominator = np.prod(one_plus)
    log_arg = numerator / denominator - 1.0 / delta

    if log_arg <= 0:
        raise RuntimeError("Projection2 failed post-opt: log_arg <= 0")

    logL0 = np.log(log_arg)
    logLpi = np.empty(N)
    logLpi[0] = logL0
    logLpi[1:] = y_opt
    return logLpi

## 3. Simulation Monte Carlo

- **Coarse check** : test rapide pour rester loin de la barrière
- **Fine check** : perturbation maximale en un pas
- **Boundary zone** : projection + saut auxiliaire

Introduce
$$ \ln L_{k, Max} = \max_i \ln L_k^i $$

$$
\ln \hat{L}_{k+1} = \ln L_{k, Max} + \sigma_{Max}^2 hN - \frac{1}{2} \sigma_{Max}^2 h + \sigma_{Max} \sqrt{hN}, \tag{6.7}
$$

where $$ \sigma_{Max} = \max_{i,k} \sigma_i(t_k).$$
Using the fact that $$ R_{swap}(\hat{L}_{k+1}, …, \hat{L}_{k+1}) = \hat{L}_{k+1} $$

### Algorithm 2.1 – weak order 1 with kick‑back reflection

In [216]:
# Nombre de chemins
n_paths    = 100_000
payoffs    = np.zeros(n_paths)
exit_times = np.zeros(n_paths)  # stocke l'étape d'exit pour chaque chemin
h = 0.25
M = int(T0 / h)           # Number of time steps until T0

# Nouvelle variable pour stocker la somme des discounts P(T0,Tj) pour chemins survivants
discount_sums = np.zeros(n_paths)
prices = np.zeros(n_paths)

for p in tqdm(range(n_paths), desc="MC paths"):
    # Initialize per-path variables
    logL = np.log(L0)
    knocked_out = False
    exit_step = M
    # Precompute coarse_bound from eq (6.7)
    sigma_max = np.max(sigma)
    coarse_bound = (
        sigma_max**2 * h * N
        - 0.5 * sigma_max**2 * h
        + sigma_max * np.sqrt(h * N)
    )

    for k in range(M):
        L = np.exp(logL)
        # Drift vector
        v         = delta * L / (1 + delta * L)
        drift_vec = drift_mat.dot(v) * h - 0.5 * sigma**2 * h

        # Coarse check: bound from (6.7)
        if np.max(logL) + coarse_bound < np.log(Rup):
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U.T @ xi)
            logL    += drift_vec + diffusion
            continue

        # Fine check: worst-case drift + diffusion per coordinate
        L_vec = L  # already computed
        idx   = np.arange(N)
        drift_worst = (idx + 1) * sigma * sigma_max * h
        diff_worst  = sigma * np.sqrt((N - idx) * h)
        L_pert      = L_vec * (1 + drift_worst + diff_worst)
        logL_pert   = np.log(L_pert)
        if R_swap(logL_pert, delta) < Rup:
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U.T @ xi)
            logL    += drift_vec + diffusion
            continue

        # Boundary zone
        logL_pi = project_to_barrier2(logL, delta, Rup)
        dist    = np.linalg.norm(logL_pi - logL)
        lambda_k = np.sqrt(N) * coarse_bound  # use bound as lambda*sqrt(h)

        if np.random.rand() < lambda_k / (dist + lambda_k):
            knocked_out = True
            exit_step   = k
            break

        # Jump back then Euler
        logL    += (lambda_k / dist) * (logL_pi - logL)
        xi        = np.random.choice([-1,1], size=N)
        diffusion = sigma * np.sqrt(h) * (U.T @ xi)
        logL    += drift_vec + diffusion

    exit_times[p] = exit_step * h
    if not knocked_out:
        payoffs[p] = delta * max(R_swap(logL, delta) - K, 0)
        # L est le vecteur des prix finaux des forwards: L_end = exp(logL)
        L_end = np.exp(logL)
        # discount_curve[j] = P(T0, T_{j+1}) = prod_{i=0..j} 1/(1+delta*L_end[i])
        discount_curve = np.cumprod(1.0/(1 + delta * L_end))
        # somme pour j=1...N
        discount_sums[p] = np.sum(discount_curve)
        prices[p] = P0_T0 * (discount_sums[p] * payoffs[p])

MC paths: 100%|██████████| 100000/100000 [02:06<00:00, 792.71it/s]


In [215]:
price_algo1      = np.mean(prices)
p_stderr_algo1   = prices.std(ddof=1) / np.sqrt(n_paths)
exit_time_algo1  = np.mean(exit_times)

print(price_algo1, exit_time_algo1)
print(price_payer)

# Header
print(f"{'h':<10} {'error':<25} {'mean exit time':<15}")
print("-" * 55)

# Row
h_value = h  # your current step size, e.g. 0.1
error = abs(price_algo1 - price_payer)
print(f"{h_value:<10.5f} "
      f"{error:.2e} ± {p_stderr_algo1:.2e}   "
      f"{exit_time_algo1:.2f}")

0.12980509569886936 8.955725
0.3088693969016971
h          error                     mean exit time 
-------------------------------------------------------
0.25000    1.79e-01 ± 8.05e-04   8.96


### Algorithm 2.2 – weak order 1/2, absorbing barrier (no VR)

In [217]:
# Nombre de chemins
n_paths    = 100_000

payoffs    = np.zeros(n_paths)
exit_times = np.zeros(n_paths)  # stocke l'étape d'exit pour chaque chemin
h = 0.25
M = int(T0 / h)           # Number of time steps until T0

# Nouvelle variable pour stocker la somme des discounts P(T0,Tj) pour chemins survivants
discount_sums = np.zeros(n_paths)
prices = np.zeros(n_paths)

for p in tqdm(range(n_paths), desc="MC paths"):
    logL        = np.log(L0)
    knocked_out = False
    exit_step   = M

    # pré-calcul constant
    sigma_max   = np.max(sigma)
    coarse_bound = (
        sigma_max**2 * h * N
        - 0.5 * sigma_max**2 * h
        + sigma_max * np.sqrt(h * N)
    )

    for k in range(M):
        L         = np.exp(logL)
        # drift Euler faible
        v         = delta * L / (1 + delta * L)
        drift_vec = drift_mat.dot(v) * h - 0.5 * sigma**2 * h

        # 1) Coarse check : si loin de la barrière
        if np.max(logL) + coarse_bound < np.log(Rup):
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U.T @ xi)
            logL    += drift_vec + diffusion
            continue

        # 2) Fine check : worst-case drift+diffusion par composante
        idx         = np.arange(N)
        drift_worst = (idx + 1) * sigma * sigma_max * h
        diff_worst  = sigma * np.sqrt((N - idx) * h)
        L_pert      = L * (1 + drift_worst + diff_worst)
        logL_pert   = np.log(L_pert)
        if R_swap(logL_pert, delta) < Rup:
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U.T @ xi)
            logL    += drift_vec + diffusion
            continue

        # 3) Zone frontière : projection & stop (ordre 1/2)
        logL        = project_to_barrier2(logL, delta, Rup)
        exit_step   = k
        knocked_out = True
        break

    exit_times[p] = exit_step * h

    if not knocked_out:
        payoffs[p]     = delta * max(R_swap(logL, delta) - K, 0)
        discounts      = np.cumprod(1.0 / (1 + delta * np.exp(logL)))
        discount_sums[p] = np.sum(discounts)
        prices[p]       = P0_T0 * discount_sums[p] * payoffs[p]

MC paths:   0%|          | 0/100000 [00:00<?, ?it/s]

MC paths: 100%|██████████| 100000/100000 [01:45<00:00, 945.76it/s]


In [218]:
price_algo2      = np.mean(prices)
p_stderr_algo2   = prices.std(ddof=1) / np.sqrt(n_paths)
exit_time_algo2  = np.mean(exit_times)

print(price_algo2, exit_time_algo2)

# Header
print(f"{'h':<10} {'error':<25} {'mean exit time':<15}")
print("-" * 55)

# Row
h_value = h  # your current step size, e.g. 0.1
error = abs(price_algo2 - price_payer)
print(f"{h_value:<10.5f} "
      f"{error:.2e} ± {p_stderr_algo2:.2e}   "
      f"{exit_time_algo2:.2f}")

0.12900116195854644 8.9415
h          error                     mean exit time 
-------------------------------------------------------
0.25000    1.80e-01 ± 2.52e-04   8.94


## 4. Rassembler tout dans une fonction

In [175]:
def algo1(L0, sigma, delta, K, Rup, T0, T_star, h, n_paths, rho, P0_T0, drift_mat, U, project_func):
    N = len(L0)
    M = int(T0 / h)

    prices = np.zeros(n_paths)
    payoffs = np.zeros(n_paths)
    exit_times = np.zeros(n_paths)
    discount_sums = np.zeros(n_paths)

    sigma_max = np.max(sigma)
    coarse_bound = (
        sigma_max**2 * h * N
        - 0.5 * sigma_max**2 * h
        + sigma_max * np.sqrt(h * N)
    )

    for p in tqdm(range(n_paths), desc="MC paths - Algo 1"):
        logL = np.log(L0)
        knocked_out = False
        exit_step = M

        for k in range(M):
            L = np.exp(logL)
            v = delta * L / (1 + delta * L)
            drift_vec = drift_mat.dot(v) * h - 0.5 * sigma**2 * h

            if np.max(logL) + coarse_bound < np.log(Rup):
                xi = np.random.choice([-1,1], size=N)
                diffusion = sigma * np.sqrt(h) * (U.T @ xi)
                logL += drift_vec + diffusion
                continue

            idx = np.arange(N)
            drift_worst = (idx + 1) * sigma * sigma_max * h
            diff_worst  = sigma * np.sqrt((N - idx) * h)
            L_pert = L * (1 + drift_worst + diff_worst)
            logL_pert = np.log(L_pert)
            if R_swap(logL_pert, delta) < Rup:
                xi = np.random.choice([-1,1], size=N)
                diffusion = sigma * np.sqrt(h) * (U.T @ xi)
                logL += drift_vec + diffusion
                continue

            # Zone frontière : projection et KO
            try:
                logL = project_func(logL, delta, Rup)
            except RuntimeError as e:
                knocked_out = True
                exit_step = k
                break

            knocked_out = True
            exit_step = k
            break

        exit_times[p] = exit_step * h

        if not knocked_out:
            R = R_swap(logL, delta)
            payoffs[p] = delta * max(R - K, 0)
            discounts = np.cumprod(1.0 / (1 + delta * np.exp(logL)))
            discount_sums[p] = np.sum(discounts)
            prices[p] = P0_T0 * discount_sums[p] * payoffs[p]

    return prices, exit_times

In [134]:
def algo2(L0, sigma, delta, K, Rup, T0, T_star, h, n_paths, rho, P0_T0, drift_mat, U, project_func):
    N = len(L0)
    M = int(T0 / h)

    prices = np.zeros(n_paths)
    payoffs = np.zeros(n_paths)
    exit_times = np.zeros(n_paths)
    discount_sums = np.zeros(n_paths)

    sigma_max = np.max(sigma)
    coarse_bound = (
        sigma_max**2 * h * N
        - 0.5 * sigma_max**2 * h
        + sigma_max * np.sqrt(h * N)
    )

    for p in tqdm(range(n_paths), desc="MC paths - Algo 2"):
        logL = np.log(L0)
        knocked_out = False
        exit_step = M

        for k in range(M):
            L = np.exp(logL)
            v = delta * L / (1 + delta * L)
            drift_vec = drift_mat.dot(v) * h - 0.5 * sigma**2 * h

            if np.max(logL) + coarse_bound < np.log(Rup):
                xi = np.random.choice([-1,1], size=N)
                diffusion = sigma * np.sqrt(h) * (U.T @ xi)
                logL += drift_vec + diffusion
                continue

            idx = np.arange(N)
            drift_worst = (idx + 1) * sigma * sigma_max * h
            diff_worst  = sigma * np.sqrt((N - idx) * h)
            L_pert = L * (1 + drift_worst + diff_worst)
            logL_pert = np.log(L_pert)
            if R_swap(logL_pert, delta) < Rup:
                xi = np.random.choice([-1,1], size=N)
                diffusion = sigma * np.sqrt(h) * (U.T @ xi)
                logL += drift_vec + diffusion
                continue

            # Zone frontière : projection puis on continue
            try:
                logL_pi = project_func(logL, delta, Rup)
                dist = np.linalg.norm(logL_pi - logL)
                lambda_k = np.sqrt(N) * coarse_bound
                if np.random.rand() < lambda_k / (dist + lambda_k):
                    knocked_out = True
                    exit_step = k
                    break

                logL += (lambda_k / dist) * (logL_pi - logL)
            except RuntimeError as e:
                knocked_out = True
                exit_step = k
                break

            xi = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U.T @ xi)
            logL += drift_vec + diffusion

        exit_times[p] = exit_step * h

        if not knocked_out:
            R = R_swap(logL, delta)
            payoffs[p] = delta * max(R - K, 0)
            discounts = np.cumprod(1.0 / (1 + delta * np.exp(logL)))
            discount_sums[p] = np.sum(discounts)
            prices[p] = P0_T0 * discount_sums[p] * payoffs[p]

    return prices, exit_times

In [176]:
prices1, exit_times1 = algo1(L0, sigma, delta, K, Rup, T0, T_star, h, n_paths, rho, P0_T0, drift_mat, U, project_to_barrier2)
prices2, exit_times2 = algo2(L0, sigma, delta, K, Rup, T0, T_star, h, n_paths, rho, P0_T0, drift_mat, U, project_to_barrier2)

MC paths - Algo 2: 100%|██████████| 10000/10000 [00:11<00:00, 838.42it/s]


In [177]:
print(np.mean(prices1), np.mean(exit_times1))
print(np.mean(prices2), np.mean(exit_times2))

0.12691380798655716 8.9027
0.13134659849099106 9.000175


In [178]:
prices1, exit_times1 = algo1(L0, sigma, delta, K, Rup, T0, T_star, h, n_paths, rho, P0_T0, drift_mat, U, project_to_barrier)
prices2, exit_times2 = algo2(L0, sigma, delta, K, Rup, T0, T_star, h, n_paths, rho, P0_T0, drift_mat, U, project_to_barrier)

MC paths - Algo 2: 100%|██████████| 10000/10000 [00:12<00:00, 809.84it/s]


In [179]:
print(np.mean(prices1), np.mean(exit_times1))
print(np.mean(prices2), np.mean(exit_times2))

0.1295923631043575 8.955125
0.13116367396410927 8.9943


# Parallélisation

In [236]:
from numba import njit, prange
import numpy as np

In [237]:
@njit
def R_swap(logL, delta):
    L = np.exp(logL)
    discount_prod = np.prod(1 + delta * L)
    numer = 1 - 1.0 / discount_prod
    denom = delta * np.sum(1.0 / np.cumprod(1 + delta * L))
    return numer / denom

In [238]:
@njit
def project_to_barrier_fast(logL, delta, Rup):
    N = len(logL)
    eps = 1e-8

    logL_tail = logL[1:]
    L_tail = np.exp(logL_tail)
    one_plus = 1.0 + delta * L_tail

    revcp = np.ones(N-1)
    revcp[-1] = one_plus[-1]
    for i in range(N-3, -1, -1):
        revcp[i] = revcp[i+1] * one_plus[i]

    sum_term = 1.0 + np.sum(revcp)
    denominator = np.prod(one_plus)

    term = (Rup * sum_term + 1.0) / denominator
    log_arg = term - 1.0 / delta
    if log_arg <= eps:
        log_arg = eps

    logL0 = np.log(log_arg)

    logLpi = np.empty(N)
    logLpi[0] = logL0
    logLpi[1:] = logL_tail

    return logLpi

In [239]:
@njit(parallel=True)
def algo1_numba(L0, sigma, delta, K, Rup, T0, T_star, h, n_paths, rho, P0_T0, drift_mat, U):
    N = len(L0)
    M = int(T0 / h)

    prices = np.zeros(n_paths)
    exit_times = np.zeros(n_paths)

    sigma_max = np.max(sigma)
    coarse_bound = (
        sigma_max**2 * h * N
        - 0.5 * sigma_max**2 * h
        + sigma_max * np.sqrt(h * N)
    )

    for p in prange(n_paths):
        logL = np.log(L0.copy())
        knocked_out = False
        exit_step = M

        for k in range(M):
            L = np.exp(logL)
            v = delta * L / (1 + delta * L)
            drift_vec = drift_mat @ v * h - 0.5 * sigma**2 * h

            if np.max(logL) + coarse_bound < np.log(Rup):
                xi = 2 * (np.random.rand(N) < 0.5) - 1
                diffusion = sigma * np.sqrt(h) * (U.T @ xi)
                logL += drift_vec + diffusion
                continue

            idx = np.arange(N)
            drift_worst = (idx + 1) * sigma * sigma_max * h
            diff_worst  = sigma * np.sqrt((N - idx) * h)
            L_pert = L * (1 + drift_worst + diff_worst)
            logL_pert = np.log(L_pert)
            if R_swap(logL_pert, delta) < Rup:
                xi = 2 * (np.random.rand(N) < 0.5) - 1
                diffusion = sigma * np.sqrt(h) * (U.T @ xi)
                logL += drift_vec + diffusion
                continue

            knocked_out = True
            exit_step = k
            break

        exit_times[p] = exit_step * h

        if not knocked_out:
            R = R_swap(logL, delta)
            payoff = delta * max(R - K, 0)
            discounts = np.cumprod(1.0 / (1 + delta * np.exp(logL)))
            discount_sum = np.sum(discounts)
            prices[p] = P0_T0 * discount_sum * payoff

    return prices, exit_times

In [285]:

@njit(parallel=True)
def algo2_numba(L0, sigma, delta, K, Rup, T0, h, n_paths, drift_mat, U, annuity):
    N = len(L0)
    M = int(T0 / h)

    prices = np.zeros(n_paths)
    exit_times = np.zeros(n_paths)

    sigma_max = np.max(sigma)

    # Coarse Bound comme dans l'article (eq 6.7)
    coarse_bound = sigma_max**2 * h * N - 0.5 * sigma_max**2 * h + sigma_max * np.sqrt(h * N)

    # Lambda_k constant
    lambda_k = np.sqrt(N) * coarse_bound

    for p in prange(n_paths):
        logL = np.log(L0.copy())
        knocked_out = False
        exit_step = M

        for k in range(M):
            L = np.exp(logL)
            v = delta * L / (1 + delta * L)
            drift_vec = drift_mat @ v * h - 0.5 * sigma**2 * h

            # Coarse check
            if np.amax(logL) + coarse_bound < np.log(Rup):
                xi = np.where(np.random.random(N) < 0.5, -1.0, 1.0)
                diffusion = sigma * np.sqrt(h) * (U.T @ xi)
                logL += drift_vec + diffusion
                continue

            # Fine check
            idx = np.arange(N)
            drift_worst = (idx + 1) * sigma * sigma_max * h
            diff_worst = sigma * np.sqrt((N - idx) * h)
            L_pert = L * (1 + drift_worst + diff_worst)
            logL_pert = np.log(L_pert)
            if R_swap(logL_pert, delta) < Rup:
                xi = np.where(np.random.random(N) < 0.5, -1.0, 1.0)
                diffusion = sigma * np.sqrt(h) * (U.T @ xi)
                logL += drift_vec + diffusion
                continue

            # Projection et rebond
            logL_pi = project_to_barrier_fast(logL, delta, Rup)
            diff_vec = logL_pi - logL
            dist = np.sqrt(np.sum(diff_vec * diff_vec) + 1e-12)

            p_accept = lambda_k / (dist + lambda_k)
            if p_accept > 1.0:
                p_accept = 1.0

            if np.random.random() < p_accept:
                knocked_out = True
                exit_step = k
                break

            logL += (lambda_k / dist) * diff_vec

            xi = np.where(np.random.random(N) < 0.5, -1.0, 1.0)
            diffusion = sigma * np.sqrt(h) * (U.T @ xi)
            logL += drift_vec + diffusion

        exit_times[p] = exit_step * h

        if not knocked_out:
            R = R_swap(logL, delta)
            payoff = delta * max(R - K, 0)
            prices[p] = delta * annuity * payoff  # PAYOFF SIMPLIFIÉ CORRECT

    return prices, exit_times

In [286]:
# Casts sécurisés :
delta = np.float64(delta)
K = np.float64(K)
Rup = np.float64(Rup)
T0 = np.float64(T0)
T_star = np.float64(T_star)
h = np.float64(h)
P0_T0 = np.float64(P0_T0)

# Casts matrices :
L0 = np.array(L0, dtype=np.float64)
sigma = np.array(sigma, dtype=np.float64)
rho = np.array(rho, dtype=np.float64)
drift_mat = np.array(drift_mat, dtype=np.float64)
U = np.array(U, dtype=np.float64)

In [287]:
# Précalcule une seule fois
P0_T = np.concatenate(([1.0], np.cumprod(1.0 / (1 + delta * L0))))
annuity = np.sum(P0_T[1:])  # somme des discounts initiaux

# Lancement simulation
prices, exit_times = algo2_numba(
    L0, sigma, delta, K, Rup, T0, h,
    1_000_000, drift_mat, U, annuity
)

In [288]:
print("h = ", h)

mean_price = np.mean(prices)
stderr_price = np.std(prices, ddof=1) / np.sqrt(len(prices))
mean_exit = np.mean(exit_times)

print(f"MC Price: {mean_price:.6f} ± {stderr_price:.2e}")
print(f"Exit time: {mean_exit:.6f}")

h =  0.03125
MC Price: 0.239244 ± 1.26e-04
Exit time: 9.412218


In [284]:
alive = (exit_times >= T0 - 1e-6)  # Chemins qui ne sont PAS KO
fraction_alive = np.mean(alive)
print(f"Fraction of alive paths: {fraction_alive:.4f}")

payoff_nonzero = (prices > 0.0)
fraction_nonzero = np.mean(payoff_nonzero)
print(f"Fraction of nonzero payoffs: {fraction_nonzero:.4f}")

print(f"Mean exit time: {np.mean(exit_times):.4f}")

Fraction of alive paths: 0.8427
Fraction of nonzero payoffs: 0.8427
Mean exit time: 9.4128


In [263]:
h = 0.03125

In [265]:
prices, exit_times = algo2_numba(
    L0, sigma, delta, K, Rup, T0, h,
    1_000_000, P0_T0, drift_mat, U
)

mean_price = np.mean(prices)
stderr_price = np.std(prices, ddof=1) / np.sqrt(len(prices))

print(f"MC Price: {mean_price:.6f} ± {stderr_price:.2e}")

MC Price: 0.147672 ± 7.38e-05


In [246]:
print(f"Mean exit time: {np.mean(exit_times):.3f}")

Mean exit time: 9.408


## 5. Résultat

Calcul du prix par moyenne des payoffs et actualisation

Cette approximation sert de référence pour évaluer la cohérence de la simulation LMM.

# 7. Comparaison avec méthode MC 1/2 ordre faible

In [54]:
for p in tqdm(range(n_paths), desc="MC paths"):
    logL        = np.log(L0)
    knocked_out = False
    exit_step   = M

    # pré-calcul constant
    sigma_max   = np.max(sigma)
    coarse_bound = (
        sigma_max**2 * h * N
        - 0.5 * sigma_max**2 * h
        + sigma_max * np.sqrt(h * N)
    )

    for k in range(M):
        L         = np.exp(logL)
        # drift Euler faible
        v         = delta * L / (1 + delta * L)
        drift_vec = drift_mat.dot(v) * h - 0.5 * sigma**2 * h

        # 1) Coarse check : si loin de la barrière
        if np.max(logL) + coarse_bound < np.log(Rup):
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U @ xi)
            logL    += drift_vec + diffusion
            continue

        # 2) Fine check : worst-case drift+diffusion par composante
        idx         = np.arange(N)
        drift_worst = (idx + 1) * sigma * sigma_max * h
        diff_worst  = sigma * np.sqrt((N - idx) * h)
        L_pert      = L * (1 + drift_worst + diff_worst)
        logL_pert   = np.log(L_pert)
        if R_swap(logL_pert, delta) < Rup:
            xi        = np.random.choice([-1,1], size=N)
            diffusion = sigma * np.sqrt(h) * (U @ xi)
            logL    += drift_vec + diffusion
            continue

        # 3) Zone frontière : projection & stop (ordre 1/2)
        logL        = project_to_barrier2(logL, delta, Rup)
        exit_step   = k
        knocked_out = True
        break

    exit_times[p] = exit_step * h

    if not knocked_out:
        payoffs[p]     = delta * max(R_swap(logL, delta) - K, 0)
        discounts      = np.cumprod(1.0 / (1 + delta * np.exp(logL)))
        discount_sums[p] = np.sum(discounts)
        prices[p]       = P0_T0 * discount_sums[p] * payoffs[p]

MC paths: 100%|██████████| 100000/100000 [03:14<00:00, 512.99it/s]


In [83]:
price           = np.mean(prices)
mean_exit_time  = np.mean(exit_times)
print(f"Barrier swaption price:    {price:.6f}")
print(f"Mean exit time (in years): {mean_exit_time:.4f}")

Barrier swaption price:    0.265078
Mean exit time (in years): 9.2737


# 8. Variance reduction